### Dynamic Programming without shortage cost 

**Instance 1:**  

    1) Use T_1 = 10
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
    
    3) Run the model 10 times to obtain 10 instances

**Instance 2:** 
    
     1) Use T_2 = 20
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
     
     3) Run the model 10 times to obtain 10 instances

**Instance 3:** 
    
     1) Use T_1 = 30
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.
     
     3) Run the model 10 times to obtain 10 instances

#### Import Modules

In [1]:
# reading file
import pandas as pd

#math and random operations
import random as rd
import math
import numpy as np

# Generating graphs
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#### Define DP Stages and States and variables

In [21]:
# rd.seed(2844) #Used for random generator seed
T_1,T_2,T_3 = 10, 20, 30 #T values for the final planning horizon period

S = 40 #storage capacity
Q = 40 # max number of units ordered per week
I0 = 0 #starting inventory of 0

# costs for each period
def generateVariables(T):
    ct = [round(rd.uniform(10,20),1) for i in range(T)] # unit ordering costs
    ht = [round(rd.uniform(5,15),1) for i in range(T)] # holding ordering costs
    dt = [int(rd.uniform(0,20)) for i in range(T)] # demand for each period 
    return [ct, ht, dt]


#### Helper functions

In [83]:
def dummy_fill_In(table, tablenum):
    if (tablenum == 1):
        table.append([i for i in range(0,Q)])
    else:    
        for row in range(0,S):
            list = []
            for col in range(0,Q):
                list.append(col)
            table.append(list)
        
def fillIn_StartBoundary(table, tablenum, dt, ht, ct):
    for col in range(0, len(table[tablenum][0])):
        if(col >= dt):
            nextI = I0 + col - dt
            cost = round(ct*col + ht*(I0+col-dt) + OptimalDec(table, 2, nextI, dt),1)
            table[tablenum][0][col] = cost
        else:
            table[tablenum][0][col] = 0
    return table

def fillIn_EndBoundary(table, tablenum, dt, ct):
    for row in range(0, len(table[tablenum])):
        for col in range(0, len(table[tablenum][row])):
            if((col + row) == dt):
                cost = round(ct * col,1)
                table[tablenum][row][col] = cost
            else:
                table[tablenum][row][col] = 0
    return table

def fillIn_Table(table, tablenum, dt, ht, ct):
    for row in range(0, len(table[tablenum])):
        minVal = max(0, dt - row)
        maxVal = min(Q, dt+S-row)
        for col in range(0, len(table[tablenum][row])):
            if(col >= minVal and col < maxVal):
                nextI = row + col - dt
#                 print(f'min={minVal}, max={maxVal}, nextI={nextI}')
                nexttable = tablenum + 1
                cost = round(ct*col + ht*(row+col-dt) + OptimalDec(table, nexttable, nextI, dt),1)
                table[tablenum][row][col] = cost
            elif(col < minVal or col > maxVal):
                table[tablenum][row][col] = 0
    return table

def OptimalDec(table, tablenum, row, dt):
    minVal = max(0, dt - row)
    maxVal = min(Q, dt+S-row)
    opt = 999
    optindex = -1
    for col in range(0,len(table[tablenum][row])):
        if(col >= minVal and col < maxVal):
            if(table[tablenum][row][col] <= opt):
                optindex = col
                opt = table[tablenum][row][col]
        else:
            continue
    return opt

def findMinInRowIndex(table, tablenum, row, dt):
    FinalCost = 99999
    indexOfFinalCost = 0
    for col in range(0,len(table[tablenum][row])):
        if((not tablenum == 1) and (not tablenum == 10)):
            if(row >= dt):
                indexOfFinalCost = 0
            else:
                if(table[tablenum][row][col] <= FinalCost):
                    FinalCost = table[tablenum][row][col]
                    indexOfFinalCost = col + 1 
        else:
            if(not table[tablenum][row][col] == 0):
                if(table[tablenum][row][col] <= FinalCost):
                    FinalCost = table[tablenum][row][col]
                    indexOfFinalCost = col
    return indexOfFinalCost

def recursive(table, tablenum, dt, a, ht, ct):
    if(tablenum == 1):
#         print(f'tablenum={tablenum}, dt={dt[a]}')
        table = fillIn_StartBoundary(table, 1, dt[a], ht, ct)
        return table
    else:
#         print(f'tablenum={tablenum}, dt={dt[a]}')
        if(tablenum == 10):
            table = fillIn_EndBoundary(table, 10, dt[a], ct)
#             dt = int(rd.uniform(0,20))
            tablenum = tablenum - 1
            a = a - 1
            return recursive(table, tablenum, dt, a, ht, ct)
        else:
            table = fillIn_Table(table, tablenum, dt[a], ht, ct)
            tablenum = tablenum - 1
            a = a - 1
            return recursive(table, tablenum, dt, a, ht, ct)

#### Solve and Run DP

In [94]:
%%time
rd.seed(2844)
# dts = []
def createDummyTables():
    Case_1 ={1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[],10:[]} 
    for i in Case_1:
        dummy_fill_In(Case_1[i], i)
    return Case_1

def RunOnce(table, T, dt, a, ht, ct):
    Case_1 = recursive(table, T, dt, a, ht, ct)
    return Case_1

var = generateVariables(T_1)

Case_1 = createDummyTables()
table = RunOnce(Case_1, T_1, var[2], 9, var[0][9], var[1][9])

n = 0
li = []
CurrInv = 0
NextInv = 0 
for t in range(1,11):
    if(t == 1):
        xVal = findMinInRowIndex(table, t, CurrInv, var[2][n])
        li.append([CurrInv, xVal, var[2][n]])
        NextInv = I0 + xVal - var[2][n]
#         print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')
    else:
        if(n == 10):
            xVal = findMinInRowIndex(table, t, CurrInv, var[2][n])
            li.append([CurrInv, xVal, var[2][n]])
            NextInv = (CurrInv + xVal - var[2][n])
#             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')
        else:
            n = n + 1
            CurrInv = NextInv
            if(CurrInv >= S):
                CurrInv = S-1
            xVal = findMinInRowIndex(table, t, CurrInv, var[2][n])
            li.append([CurrInv, xVal, var[2][n]])
            NextInv = (CurrInv + xVal - var[2][n])
#             print(f'table={t}, CurrInv={CurrInv}, NextInv={NextInv}, Col={xVal}, n={n}, dt={var[2][n]}')

print(li,'\n',ct[9], ht[9],'\n')

# Generate Average + Standard Deviation over all the 10 instances, prodcution cost, holding cost, runtime
totCost = []
prodCost = []
holdCost = []
tables = []
AverageTotalCost = []
CurrentInv= []
UnitsProd = []
Demand = []
for i in range(0, len(li)):
    CurrentInv.append(li[i][0])
    UnitsProd.append(li[i][1])
    Demand.append(li[i][2])

for i in range(1, 11): #test instance 1 use 11 and st_1, test instance 2 use 21 and st_2, test instance 3 use 31 and st_3
    tables.append(i)
#     totCost.append(table[i][li[i-1][0]][li[i-1][1]])
    pc = round(ct[9]*li[i-1][1],1)
    prodCost.append(pc)
    hc = round(ht[9]*(li[i-1][0]+li[i-1][1]-li[i-1][2]),1)
    holdCost.append(hc)
    totCost.append(pc + hc)
df = pd.DataFrame({
        'Period':tables,
        'Current Inventory':CurrentInv,
        'Units Produced':UnitsProd,
        'Demand':Demand,
        'Production Cost' : prodCost,
        'Holding Cost' : holdCost,
        'Total Cost In Period' : totCost
    })

SumCost = df.sum(axis=0)
AvgProdCost = df.mean(axis=0)
df

[[0, 4, 4], [0, 4, 4], [0, 16, 16], [0, 18, 18], [0, 0, 0], [0, 7, 7], [0, 14, 14], [0, 7, 7], [0, 7, 7], [0, 7, 7]] 
 10.8 13.0 

Wall time: 92.3 ms


,Period,Current Inventory,Units Produced,Demand,Production Cost,Holding Cost,Total Cost In Period
0,1,0,4,4,43.2,0.0,43.2
1,2,0,4,4,43.2,0.0,43.2
2,3,0,16,16,172.8,0.0,172.8
3,4,0,18,18,194.4,0.0,194.4
4,5,0,0,0,0.0,0.0,0.0
5,6,0,7,7,75.6,0.0,75.6
6,7,0,14,14,151.2,0.0,151.2
7,8,0,7,7,75.6,0.0,75.6
8,9,0,7,7,75.6,0.0,75.6
9,10,0,7,7,75.6,0.0,75.6


#### Get Averages and Total Cost

In [86]:
print(f'Average Production Cost = {round(AvgProdCost[4],1)} \nTotal Cost = {round(SumCost[6],1)}')

Average Production Cost = 109.1 
Total Cost = 2689.8


### Test Cases

### Instance 1